In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
import time
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from IPython.display import display
from PIL import Image

# Predicting for unique views at 180 days
Use Google Anaytics to obtain unique views at 180 days and calculate binary
0 = no view
1 = view

In [ ]:
df = pd.read_csv("File.csv")
df.info()
df.head()

In [ ]:
deep_df = df.copy(deep = True)

In [ ]:
numerical_columns = [col for col in df.columns if (df[col].dtype=='int64' or df[col].dtype=='float64') and col != 'Exited']

In [ ]:
df[numerical_columns].describe().loc[['min','max', 'mean','50%'],:]

In [ ]:
df.head()

In [ ]:
df[df['iContact_messageId'] == df['iContact_messageId'].min()]

In [ ]:
feat = df.drop(columns=['Month'],axis=1)
label = df["Month"]

In [ ]:
##clean textual data

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
%%time
wordnet_lemmatizer = WordNetLemmatizer()
stopset = list(set(stopwords.words('english')))
clean_reviews_text = []
for review in df['ArticleText']:  # Loop through the tokens (the words or symbols) in each review. 
    try:    
        cleaned_review = re.sub("[^a-zA-Z]"," ", review)  # Remove numbers and punctuation.
        cleaned_review = cleaned_review.lower()  # Convert the text to lower case.
        cleaned_review = ' '.join([word for word in cleaned_review.split() if word not in stopset])  # Keep only words that are not stopwords.
        cleaned_review = ' '.join([wordnet_lemmatizer.lemmatize(word, pos='n') for word in cleaned_review.split()])  # Keep each noun's lemma.
        cleaned_review = ' '.join([wordnet_lemmatizer.lemmatize(word, pos='v') for word in cleaned_review.split()])  # Keep each verb's lemma.
        cleaned_review = re.sub(r"(http\S+)"," ", cleaned_review)  # Remove http links.
        cleaned_review = ' '.join(cleaned_review.split())  # Remove white space.
    except TypeError:
        pass
    clean_reviews_text.append(cleaned_review)

In [ ]:
##show dataframe with cleanText column added

In [ ]:
df['cleanText'] = clean_reviews_text
df.head()

In [ ]:
from wordcloud import WordCloud
import nltk

In [ ]:
one_long_string =  ' '.join(df['cleanText'])

In [ ]:
##plot word cloud of all texts
wordcloud = WordCloud(background_color='silver',max_font_size = 50,random_state=75, relative_scaling=.3, collocations=True).generate(one_long_string)
plt.figure(figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('All texts')
plt.show()

In [ ]:
##create dataframe of unique views and no unique views, set views goal based on company unique view goals
df_noviews = df[df['uniqueviews'] == 0]
df_views = df[df['uniqueviews'] == 1]

In [ ]:
one_long_string_noviews =  ' '.join(df_noviews['cleanText'])
one_long_string_views =  ' '.join(df_views['cleanText'])

In [ ]:
wordcloud_noviews = WordCloud(background_color='silver',max_font_size = 50,random_state=201).generate(one_long_string_noviews)
wordcloud_views = WordCloud(background_color='silver',max_font_size = 50,random_state=201).generate(one_long_string_views)

plt.figure(figsize=(15, 15))

plt.subplot(121)  # 121 means 1 row and 2 columns of plots and this is the first subplot.
plt.imshow(wordcloud_noviews)
plt.axis("off")
plt.title('Unique Views at 180 Days 750 or Below') 

plt.subplot(122)  # 122 means 1 row and 2 columns of plots and this is the second subplot.
plt.imshow(wordcloud_views)
plt.axis("off")
plt.title('Unique Views at 180 Days above 750')

plt.show()

In [ ]:
##show bag of words dataframe with word usage
corpus = list(df['cleanText'])
corpus_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=30)
bag_of_words = corpus_vectorizer.fit_transform(corpus)
bag_of_words_df = pd.DataFrame(bag_of_words.toarray(), columns=corpus_vectorizer.get_feature_names())
bag_of_words_df

In [ ]:
pd.DataFrame(bag_of_words_df.mean(axis=0), index=bag_of_words_df.columns, columns=['Avg count (all texts)'])

In [ ]:
corpus_noviews = list(df_noviews['cleanText'])
corpus_noviews_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20)
bag_of_words_noviews = corpus_noviews_vectorizer.fit_transform(corpus_noviews)
bag_of_words_noviews_df = pd.DataFrame(bag_of_words_noviews.toarray(), columns=corpus_noviews_vectorizer.get_feature_names())
bag_of_words_noviews_df

In [ ]:
corpus_views = list(df_views['cleanText'])
corpus_views_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20)
bag_of_words_views = corpus_views_vectorizer.fit_transform(corpus_views)
bag_of_words_views_df = pd.DataFrame(bag_of_words_views.toarray(), columns=corpus_views_vectorizer.get_feature_names())
bag_of_words_views_df

In [ ]:
freq_words_noviews = pd.DataFrame(bag_of_words_noviews_df.mean(axis=0), 
                                     index=bag_of_words_noviews_df.columns, columns=['Avg count (noviews)'])
freq_words_views = pd.DataFrame(bag_of_words_views_df.mean(axis=0), 
                                     index=bag_of_words_views_df.columns, columns=['Avg count (views)'])

In [ ]:
freq_words_noviews.join(freq_words_views, how='outer')

In [ ]:
##load a happiness dictionary

In [ ]:
import json
import urllib.request
url='http://hedonometer.org/api/v1/words/?format=json'
data = urllib.request.urlopen(url).read().decode('utf-8')
loaded_json = json.loads(data)
happ_dict = loaded_json['objects']
from pandas.io.json import json_normalize
happ_df = json_normalize(happ_dict)

In [ ]:
word_happs_df = happ_df[['word', 'happs']]

In [ ]:
word_happs_df.shape

In [ ]:
##load the Harvard General Inquirer Dictionary

In [ ]:
genInq_df = pd.read_csv('inquirerbasic.csv', encoding='latin-1', low_memory=False)

In [ ]:
genInq_df.head()

In [ ]:
genInq_df['Entry'] = [word.lower() for word in genInq_df['Entry']]

In [ ]:
summary_df = pd.concat([pd.DataFrame(genInq_df.columns), pd.DataFrame(genInq_df.dtypes.values.reshape(-1,1)),
                        pd.DataFrame(genInq_df.isnull().sum().values), pd.DataFrame([genInq_df[name].nunique() for name in genInq_df.columns])],
                       axis=1)
summary_df.columns = ['Variable Name', 'Data Type', 'Nulls', 'Unique Values']

In [ ]:
summary_df[0:100]

In [ ]:
##do some feature engineering

In [ ]:
df = df.reset_index(drop=True)  # First reset the index of the data frame. Drops first column so that don't interfere with data

In [ ]:
%%time
wildfire_dummy = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    wildfire_dummy.append(int('go' in set(text.split())))
df['contains_wildfire'] = wildfire_dummy

In [ ]:
%%time
happs_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['word'])
    text_happs_df = pd.merge(text_df, word_happs_df, on='word')
    happs_list.append(text_happs_df['happs'].sum())
df['happ_score'] = happs_list

In [ ]:
postivie_dummy_df = pd.get_dummies(genInq_df[['Entry','Positiv']], prefix=['contains'], columns=['Positiv'])
postivie_dummy_df.head()

In [ ]:
%%time
positive_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['Entry'])
    text_positive_df = pd.merge(text_df, postivie_dummy_df, on='Entry')
    positive_list.append(text_positive_df['contains_Positiv'].sum())
df['positive_score'] = positive_list

In [ ]:
df.tail()

In [ ]:
%%time
risk_dummy = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    risk_dummy.append(int('risk' in set(text.split())))
df['contains_risk'] = risk_dummy

In [ ]:
%%time
happs_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['word'])
    text_happs_df = pd.merge(text_df, word_happs_df, on='word')
    happs_list.append(text_happs_df['happs'].sum())
df['happ_score'] = happs_list

In [ ]:
postivie_dummy_df = pd.get_dummies(genInq_df[['Entry','Positiv']], prefix=['contains'], columns=['Positiv'])
postivie_dummy_df.head()

In [ ]:
%%time
positive_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['Entry'])
    text_positive_df = pd.merge(text_df, postivie_dummy_df, on='Entry')
    positive_list.append(text_positive_df['contains_Positiv'].sum())
df['positive_score'] = positive_list

In [ ]:
df.tail()

In [ ]:
%%time
land_dummy = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    land_dummy.append(int('risk' in set(text.split())))
df['contains_land'] = land_dummy

In [ ]:
%%time
happs_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['word'])
    text_happs_df = pd.merge(text_df, word_happs_df, on='word')
    happs_list.append(text_happs_df['happs'].sum())
df['happ_score'] = happs_list

In [ ]:
postivie_dummy_df = pd.get_dummies(genInq_df[['Entry','Positiv']], prefix=['contains'], columns=['Positiv'])
postivie_dummy_df.head()

In [ ]:
%%time
positive_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['Entry'])
    text_positive_df = pd.merge(text_df, postivie_dummy_df, on='Entry')
    positive_list.append(text_positive_df['contains_Positiv'].sum())
df['positive_score'] = positive_list

In [ ]:
df.tail()

In [ ]:
%%time
economic_dummy = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    economic_dummy.append(int('economic' in set(text.split())))
df['contains_economic'] = economic_dummy

In [ ]:
%%time
happs_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['word'])
    text_happs_df = pd.merge(text_df, word_happs_df, on='word')
    happs_list.append(text_happs_df['happs'].sum())
df['happ_score'] = happs_list

In [ ]:
postivie_dummy_df = pd.get_dummies(genInq_df[['Entry','Positiv']], prefix=['contains'], columns=['Positiv'])
postivie_dummy_df.head()

In [ ]:
%%time
positive_list = []
for i in range(0, len(df)):
    text = df.loc[i]['cleanText']
    text_df = pd.DataFrame(pd.Series(text.split()), columns=['Entry'])
    text_positive_df = pd.merge(text_df, postivie_dummy_df, on='Entry')
    positive_list.append(text_positive_df['contains_Positiv'].sum())
df['positive_score'] = positive_list

In [ ]:
df.tail()

In [ ]:
##split into training and testing sets

In [ ]:
ind_variables_selected = ['contains_economic','happ_score', 'positive_score']

In [ ]:
X_orig_train, X_test, y_orig_train, y_test = train_test_split(df[ind_variables_selected], df['uniqueviews'], test_size=0.25, random_state=201)

In [ ]:
np.mean(y_orig_train)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_orig_train, y_orig_train, test_size=0.2, random_state=201)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
y_train.head()

In [ ]:
##set up scoring rule

In [ ]:
np.mean(y_train)

In [ ]:
def brier_score(predictions, realizations):
    this_brier_score = 1 - brier_score_loss(realizations, predictions)
    return this_brier_score

In [ ]:
def skill_score(predictions, realizations):
    naive = np.repeat(np.mean(y_train), len(realizations))
    this_skill_score = (brier_score(predictions, realizations) - brier_score(naive, realizations)) / (1 - brier_score(naive, realizations))
    return this_skill_score

In [ ]:
##fit a classification tree

In [ ]:
dt = DecisionTreeClassifier(min_samples_split=5000, max_depth=50, random_state=201)
dt_model = dt.fit(X_train, y_train)

In [ ]:
pd.DataFrame(dt_model.feature_importances_, index=ind_variables_selected).sort_values(by = 0)

In [ ]:
dt_test_prob_all = pd.DataFrame(dt_model.predict_proba(X_valid))
dt_test_prob = dt_test_prob_all[1]

In [ ]:
skill_score(dt_test_prob, y_valid)

In [ ]:
dt_model = dt.fit(X_orig_train, y_orig_train)
dt_test_prob_all = pd.DataFrame(dt_model.predict_proba(X_test))
dt_test_prob = dt_test_prob_all[1]
skill_score(dt_test_prob, y_test)

In [ ]:
from sklearn.svm import SVC

In [ ]:
support_vector_classifier = SVC(kernel='rbf')
support_vector_classifier.fit(X_train,y_train)
y_pred_svc = support_vector_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm_support_vector_classifier = confusion_matrix(y_test,y_pred_svc)
print(cm_support_vector_classifier,end='\n\n')

In [ ]:
numerator = cm_support_vector_classifier[0][0] + cm_support_vector_classifier[1][1]
denominator = sum(cm_support_vector_classifier[0]) + sum(cm_support_vector_classifier[1])
acc_svc = (numerator/denominator) * 100
print("Accuracy : ",round(acc_svc,2),"%")

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_svc = cross_val_score(estimator = SVC(kernel = 'rbf'), X = X_train, y = y_train, cv = 10, n_jobs = -1)
print("Cross Validation Accuracy : ",round(cross_val_svc.mean() * 100 , 2),"%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_train,y_train)
y_pred_rfc = random_forest_classifier.predict(X_test)

In [ ]:
cm_random_forest_classifier = confusion_matrix(y_test,y_pred_rfc)
print(cm_random_forest_classifier,end="\n\n")

In [ ]:
numerator = cm_random_forest_classifier[0][0] + cm_random_forest_classifier[1][1]
denominator = sum(cm_random_forest_classifier[0]) + sum(cm_random_forest_classifier[1])
acc_rfc = (numerator/denominator) * 100
print("Accuracy : ",round(acc_rfc,2),"%")

In [ ]:
cross_val_rfc = cross_val_score(estimator=RandomForestClassifier(), X=X_train, y=y_train, cv=10, n_jobs=-1)
print("Cross Validation Accuracy : ",round(cross_val_rfc.mean() * 100 , 2),"%")

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [ ]:
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train,y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

In [ ]:
cm_xgb_classifier = confusion_matrix(y_test,y_pred_xgb)
print(cm_xgb_classifier,end='\n\n')

In [ ]:
numerator = cm_xgb_classifier[0][0] + cm_xgb_classifier[1][1]
denominator = sum(cm_xgb_classifier[0]) + sum(cm_xgb_classifier[1])
acc_xgb = (numerator/denominator) * 100
print("Accuracy : ",round(acc_xgb,2),"%")

In [ ]:
cross_val_xgb = cross_val_score(estimator=XGBClassifier(), X=X_train, y=y_train, cv=10, n_jobs=-1)
print("Cross Validation Accuracy : ",round(cross_val_xgb.mean() * 100 , 2),"%")